# [ CNN ] OOD Detection using FSS

In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.append(os.getcwd() + '/core')
sys.path.append(os.getcwd() + '/core/train_GLOW') 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

import core.config as config
from core.model_loader import load_pretrained_CNN
from core.data_loader import TRAIN_loader, TEST_loader
from core.fisher_utils_CNN import Calculate_fisher_CNN, Calculate_score_CNN
from core.fisher_utils_CNN import AUTO_CNN
from core.visualize import plot_hist, AUROC, plot_scores_all_layers

# Define Global Variables & Initialize

In [2]:
# FISHER_invs : Fisher inverse matrices from calculating Fisher score (w.r.t. train-dist)
FISHERs = {'cifar10': {}, 'fmnist': {}}
         
# NORMalize_FACtors : Normalizing Factor (w.r.t. train-dist)
NORM_FACs = {'cifar10': {}, 'fmnist': {}}
             
# SCOREs : (Scalars) Scores from calculating Fisher score (w.r.t. target-dist)
SCOREs = {'cifar10': {}, 'fmnist': {}}


# CNN-FMNIST

In [183]:
acc = 0
loader = TEST_loader('fmnist', 'fmnist')
for i, (x, y) in enumerate(loader):
    pred = model(x.float().cuda())
    if torch.argmax(pred).detach().cpu() == y:
        acc += 1
    print(f'Classification Accuracy : {acc / (i+1):.4f}', end='\r')
    if i == 10000:
        break

In [177]:
opt = config.CNN_fmnist
model = load_pretrained_CNN(option=opt.train_dist)
model.eval()

layers = {
    'conv1': model.layer1[0],
    'conv2': model.layer2[0],
    'conv3': model.layer3[0],
    'fc1': model.fc1,
    'conv4': model.layer4[0],
    'fc2': model.fc2,
}

In [175]:
a, b, _scores = AUTO_CNN(opt, model, layers, max_iter=[10000, 5000], method='Vanilla')

Calculate Fisher CNN:   0%|                                                                | 0/60000 [00:02<?, ?step/s]


AttributeError: 'set' object has no attribute 'items'

In [3]:
opt = config.VAE_cifar10
netE, netG = load_pretrained_VAE(option=opt.train_dist, ngf=64, nz=200, beta=1)
netE.eval()
netG.eval()

params = {
    #'Emain0_w': netE.main[0].weight,
    #'Emain1_w': netE.main[1].weight,
    #'Emain1_b': netE.main[1].bias,
    #'Emain3_w': netE.main[3].weight,
    #'Emain4_w': netE.main[4].weight,
    #'Emain4_b': netE.main[4].bias,
    #'Emain6_w': netE.main[6].weight,
    #'Emain7_w': netE.main[7].weight,
    #'Emain7_b': netE.main[7].bias,
    'Econv1_w': netE.conv1.weight,
    #'Econv1_b': netE.conv1.bias,
    #'Econv2_w': netE.conv2.weight,
    #'Econv2_b': netE.conv2.bias,
    #'Gmain0_w': netG.main[0].weight,
    #'Gmain1_w': netG.main[1].weight,
    #'Gmain1_b': netG.main[1].bias,
    #'Gmain3_w': netG.main[3].weight,
    #'Gmain4_w': netG.main[4].weight,
    #'Gmain4_b': netG.main[4].bias,
    #'Gmain6_w': netG.main[6].weight,
    #'Gmain7_w': netG.main[7].weight,
    #'Gmain7_b': netG.main[7].bias,
    #'Gmain9_w': netG.main[9].weight,
}


In [5]:
a, b, _scores = AUTO_VAE(opt, netE, netG, params, max_iter=[3000, 3000], loss_type='ELBO_pixel', method='SMW')

Files already downloaded and verified


Calculate Fisher VAE:   0%|                                                      | 4/50000 [00:03<9:22:34,  1.48step/s]

torch.Size([800, 1024, 1024])
torch.Size([800, 1024, 1024])
torch.Size([800, 1024, 1024])
torch.Size([800, 1024, 1024])
torch.Size([800, 1024, 1024])


Calculate Fisher VAE:   0%|                                                     | 10/50000 [00:03<2:51:00,  4.87step/s]

torch.Size([800, 1024, 1024])
torch.Size([800, 1024, 1024])
torch.Size([800, 1024, 1024])
torch.Size([800, 1024, 1024])
torch.Size([800, 1024, 1024])
torch.Size([800, 1024, 1024])


...

torch.Size([800, 1024, 1024])
torch.Size([800, 1024, 1024])
torch.Size([800, 1024, 1024])
torch.Size([800, 1024, 1024])
torch.Size([800, 1024, 1024])
torch.Size([800, 1024, 1024])


Calculate Fisher VAE:   0%|                                                       | 79/50000 [00:06<31:55, 26.06step/s]

torch.Size([800, 1024, 1024])
torch.Size([800, 1024, 1024])
torch.Size([800, 1024, 1024])
torch.Size([800, 1024, 1024])


Calculate Fisher VAE:   0%|                                                     | 81/50000 [00:06<1:07:20, 12.35step/s]


KeyboardInterrupt: 

In [7]:
for pname in params.keys():
    FISHERs[train_dist][pname] = a[pname] 
    NORM_FACs[train_dist][pname] = b[pname]
for ood in opt.ood_list:
    SCOREs[train_dist][ood] = _scores[ood]

In [8]:
# AUROC curve
auroc = {}
for pname in params.keys():
    _auroc = {}
    for ood in opt.ood_list:
        args = [
            SCOREs[train_dist][train_dist][pname],
            SCOREs[train_dist][ood][pname],
        ]
        labels = [train_dist, ood]
        _auroc[ood] = AUROC(*args, labels=labels, verbose=False)
    auroc[pname] = _auroc
    
auroc

In [6]:
# Just show scores
# plot_scores_all_layers(train_dist, params, SCOREs, opt, save=True)


# VAE-FMNIST

In [8]:
opt = config.VAE_fmnist
netE, netG = load_pretrained_VAE(option=opt.train_dist, ngf=32, nz=100, beta=1)
netE.eval()
netG.eval()

params = {
    #'Emain0_w': netE.main[0].weight,
    #'Emain1_w': netE.main[1].weight,
    #'Emain1_b': netE.main[1].bias,
    #'Emain3_w': netE.main[3].weight,
    #'Emain4_w': netE.main[4].weight,
    #'Emain4_b': netE.main[4].bias,
    #'Emain6_w': netE.main[6].weight,
    #'Emain7_w': netE.main[7].weight,
    #'Emain7_b': netE.main[7].bias,
    'Econv1_w': netE.conv1.weight,
    #'Econv1_b': netE.conv1.bias,
    #'Econv2_w': netE.conv2.weight,
    #'Econv2_b': netE.conv2.bias,
    #'Gmain0_w': netG.main[0].weight,
    #'Gmain1_w': netG.main[1].weight,
    #'Gmain1_b': netG.main[1].bias,
    #'Gmain3_w': netG.main[3].weight,
    #'Gmain4_w': netG.main[4].weight,
    #'Gmain4_b': netG.main[4].bias,
    #'Gmain6_w': netG.main[6].weight,
    #'Gmain7_w': netG.main[7].weight,
    #'Gmain7_b': netG.main[7].bias,
    #'Gmain9_w': netG.main[9].weight,
}

RuntimeError: Error(s) in loading state_dict for DCGAN_G:
	Missing key(s) in state_dict: "main.initial:100-128:convt.weight", "main.initial:128:batchnorm.weight", "main.initial:128:batchnorm.bias", "main.initial:128:batchnorm.running_mean", "main.initial:128:batchnorm.running_var", "main.pyramid:64-32:convt.weight", "main.pyramid:32:batchnorm.weight", "main.pyramid:32:batchnorm.bias", "main.pyramid:32:batchnorm.running_mean", "main.pyramid:32:batchnorm.running_var", "main.final:32-1:convt.weight". 
	Unexpected key(s) in state_dict: "main.initial:200-256:convt.weight", "main.initial:256:batchnorm.weight", "main.initial:256:batchnorm.bias", "main.initial:256:batchnorm.running_mean", "main.initial:256:batchnorm.running_var", "main.initial:256:batchnorm.num_batches_tracked", "main.pyramid:256-128:convt.weight", "main.pyramid:128:batchnorm.weight", "main.pyramid:128:batchnorm.bias", "main.pyramid:128:batchnorm.running_mean", "main.pyramid:128:batchnorm.running_var", "main.pyramid:128:batchn

In [ ]:
a, b, _scores = AUTO_VAE(opt, netE, netG, params, max_iter=[3000, 3000], loss_type='ELBO_pixel', method='SMW')

In [ ]:
for pname in params.keys():
    FISHERs[train_dist][pname] = a[pname] 
    NORM_FACs[train_dist][pname] = b[pname]
for ood in opt.ood_list:
    SCOREs[train_dist][ood] = _scores[ood]

In [ ]:
# AUROC curve
auroc = {}
for pname in params.keys():
    _auroc = {}
    for ood in opt.ood_list:
        args = [
            SCOREs[train_dist][train_dist][pname],
            SCOREs[train_dist][ood][pname],
        ]
        labels = [train_dist, ood]
        _auroc[ood] = AUROC(*args, labels=labels, verbose=False)
    auroc[pname] = _auroc
    
auroc

In [ ]:
# Just show scores
# plot_scores_all_layers(train_dist, params, SCOREs, opt, save=True)